# Part 1 - WML Federated Learning with MNIST for Admin 

With IBM Federated Learning, you can combine data from multiple sources to train a model from the collective data without having to actually share them. This allows enterprises to train data with other companies without delegating resources for security. Another advantage is the remote data does not have to be centralized in one location, eliminates the needs to move potentially large datasets. This notebook demonstrates how to start Federated Learning with the Python client. For more details setting up Federated Learning, terminology, and running Federated Learning from the UI, see [Federated Learning documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fed-lea.html?audience=wdp).

### Learning Goals

When you complete the Part 1 - WML Federated Learning with MNIST for Admin notebook, you should know how to:

- Load an untrained model
- Create a Remote Training System
- Start a training job

Once you complete this notebook, please open Part 2 - WML Federated Learning with MNIST for Party. 

<div class="alert alert-block alert-info">This notebook is intended to be run by the administrator of the Federated Learning experiment.</div>

## Table of Contents

- [1. Prequisites](#prequisites)
    - [1.2 Define variables](#var)
    - [1.3 Import libraries](#libraries)
    - [1.3 Obtain Cloud authentication token](#cloud-auth)
- [2. Load the model](#load)
- [3. Create a Remote Training System](#rts)
- [4. Create FL Training Job](#fl-job)
    - [4.1 Get Training Job Status](#status)

<a id = "prequisites"></a>
## 1. Prequisites

In this sample project, the model template and untrained datasets are provided for you. The only thing that you need to provide is the credentials, as follows.

<a id = "var"></a>
### 1.1 Define variables

In [1]:
API_VERSION = "2019-10-25"

# hostname of CP4D cluster
CP4D_HOST = # <host URL>
CP4D_URL = "https://" + CP4D_HOST

# CP4D user
WS_USER = # <username>
WS_PASSWORD = # <password>
# user id for admin user, get this in Administration < User Management (CP4D_URL/zen-admin/?deployment_target=icp4data#/usermgmt-ui)
WS_USERID = # <user ID>

# project id where this notebook resides and model id you uploaded earlier. 
PROJECT_ID = "" # You can get this by looking in your project URL

<a id = "libraries"></a>
### 1.2 Import libraries

In [2]:
import urllib3
import requests
import json
from string import Template

urllib3.disable_warnings()

<a id = "cloud-auth"></a>
### 1.3 Obtain Cloud authentication token

In [3]:
import base64

ws_userpass = WS_USER + ":" + WS_PASSWORD
enc_bytes = base64.b64encode(ws_userpass.encode("utf-8"))
ws_auth = str(enc_bytes, "utf-8")

token_resp = requests.get(CP4D_URL + "/v1/preauth/validateAuth",
                          headers={"Content-Type": "application/json",
                                   "Authorization": "Basic " + ws_auth},
                          verify=False)

In [ ]:
print("Token response: " + token_resp.content.decode("utf-8"))
token = "Bearer " + json.loads(token_resp.content.decode("utf-8"))["accessToken"]
print("WS Token: %s " % token)

<a id = "load"></a>
## 2. Load the model

You need an untrained model asset for Federated Learning to work with. In this tutorial, an untrained Tensorflow 2 Keras model is provided for you. It is named MNIST Model and comes with this tutorial demo project. Federated Learning supports Scikit-learn and Tensorflow 2, which are free machine learning packages with tutorials. Additionally IBM docs provide some details on how to configure an untrained model for Federated Learning. See:

- [Scikit-learn model configuration](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fl-imp-det.html?audience=wdp&context=cpdaas#sklearn-config)
- [Tensorflow 2 model configuration](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fl-imp-det.html?audience=wdp&context=cpdaas#tf-config)

### Create initial model asset

In [ ]:
base_model_save_payload = Template("""
{
    "name": "Untrained MNIST Model",
    "type": "tensorflow_2.1 ",
    "software_spec": {
        "name": "default_py3.7"
    },
    "custom": {
        "untrained" : true
    },
    "project_id": "$projectId"
}
""").substitute(projectId = PROJECT_ID)

print ("Model base save payload: %s" % base_model_save_payload)

base_model_save_resp = requests.post(CP4D_URL + "/ml/v4/models",
                                     headers={"Content-Type": "application/json",
                                              "Authorization": token},
                                     params={"version": API_VERSION},
                                     data=base_model_save_payload,
                                     verify=False)

print("Base model save response " + base_model_save_resp.content.decode("utf-8"))

In [ ]:
base_model_id = json.loads(base_model_save_resp.content.decode("utf-8"))["metadata"]["id"]
print("Saved model id: %s" % base_model_id)

### Upload untrained model

In [ ]:
MODEL_URL = CP4D_URL + "/ml/v4/models/"+ base_model_id + "/content"
print("Model URL: " + MODEL_URL)



with open('/project_data/data_asset/tf_mnist_model.zip', 'rb') as data:
  upload_resp = requests.put(MODEL_URL,
                             headers={"Content-Type": "application/json",
                                      "Authorization": token},
                             params={"project_id": PROJECT_ID,
                                     "version": API_VERSION,
                                     "content_format": "native"},
                             data=data,
                             verify=False)



print("Response status for upload base model  " + base_model_id + " : "+ str(upload_resp.status_code))
print("Create library content: %s"  % upload_resp.content.decode("utf-8"))

<a id = "rts"></a>
## 3. Create a Remote Training System

Now you will learn to create a Remote Training System (RTS). An RTS handles receiving your multiple parties' call to the aggregator to run the training. 

- `allowed_identities` are users permitted to connect to the Federated Learning experiment.  In this tutorial, only your user ID is permitted to connect but you can update the template and add additional users as required.
- An Admin in `remote_admin`. The template for the admin is the same as the user. In this tutorial, a template Admin is created. It is also the same as the user ID, however generally in application, the admin does not have to be one of the users.

In [8]:
wml_remote_training_system_asset_one_def = Template("""
{
  "name": "Remote Party 1",
  "project_id": "$projectId",
  "description": "Sample Remote Training System",
  "tags": [ "Federated Learning" ],
  "organization": {
    "name": "IBM",
    "region": "US"
  },
  "allowed_identities": [
    {
      "id": "$userID",
      "type": "user"
    }
  ],
  "remote_admin": {
    "id": "$userID",
    "type": "user"
  }
}
""").substitute(userID = WS_USERID,
                projectId = PROJECT_ID)


wml_remote_training_system_one_resp = requests.post(CP4D_URL + "/ml/v4/remote_training_systems", 
                                                    headers={"Content-Type": "application/json",
                                                             "Authorization": token}, 
                                                    params={"version": API_VERSION,
                                                            "project_id": PROJECT_ID}, 
                                                    data=wml_remote_training_system_asset_one_def, 
                                                    verify=False)

In [ ]:
print(wml_remote_training_system_one_resp)
status_json = json.loads(wml_remote_training_system_one_resp.content.decode("utf-8"))
print("Create wml_remote_training_system_one asset response: %s"  % json.dumps(status_json, indent=4))
wml_remote_training_system_one_asset_uid = status_json["metadata"]["id"]
print("WML wml_remote_training_system_one asset uid: %s" % wml_remote_training_system_one_asset_uid)

<a id = "fl-job"></a>
## 4. Create FL Training Job

In this section, you will launch the Federated Learning experiment.

In [ ]:
training_payload = Template(""" 
{
  "name": "FL Aggregator",
  "tags": [
    {
      "value": "tags_jobs_fl",
      "description": "Sample FL Aggregator"
    }
  ],
  "federated_learning": {
    "model": {
      "spec": {
        "id": "$modelID"
      },
      "type": "tensorflow"
    },
    "fusion_type": "iter_avg",
    "rounds": 5,
    "remote_training" : {
      "quorum": 1.0,
      "remote_training_systems": [ { "id" : "$rts_one", "required" : true  } ]
    },
    "hardware_spec": {
      "name": "XS"
    }
  },
  "training_data_references": [],
  "results_reference": {
    "type": "fs",
    "name": "outputData",
    "location": {
      "path": "projects/$projectId"
    }
  },
  "project_id": "$projectId"  
}
""").substitute(modelID = base_model_id,
                projectId = PROJECT_ID,
                rts_one = wml_remote_training_system_one_asset_uid,
                )

print("Training payload: %s" % training_payload)

training_launch_resp = requests.post(CP4D_URL + "/ml/v4/trainings", 
                                     headers={"Content-Type": "application/json",
                                              "Authorization": token},
                                     data=training_payload,
                                     verify=False)

print(training_launch_resp)
print("Launch training response %s"  % training_launch_resp.content.decode("utf-8"))
training_id = json.loads(training_launch_resp.content.decode("utf-8"))["metadata"]["id"]
print("Training ID: %s" % training_id)

<a id = "status"></a>
### 4.1 Get Training Job Status



In [ ]:
status_full = requests.get(CP4D_URL + "/ml/v4/trainings/" + training_id,
                           headers={"Content-Type": "application/json",
                                    "Authorization": token},
                           params={"version": API_VERSION,
                                   "project_id": PROJECT_ID},
                           verify=False)

print(status_full)
status_json = json.loads(status_full.content.decode("utf-8"))
print("Full training job status: "+ json.dumps(status_json, indent=4))

### Wait For Aggregator Status To Become Accepting Parties

<div class="alert alert-block alert-warning">Before you proceed further, run the previous cell. The output should be `accepting_parties`. If output is `pending`, wait for a few minutes and re-run the <i>previous</i> code cell, with <code>requests.get</code>
</div>

### Get Variables And Paste Into Party Notebook

In [ ]:
print("CP4D_HOST = \"%s\"" % CP4D_HOST)
print("CP4D_URL = \"%s\"" % CP4D_URL)
print("WS_USER = \"%s\"" % WS_USER)
print("WS_PASSWORD = \"%s\"" % WS_PASSWORD)
print("TRAINING_ID = \"%s\"" % training_id)
print("RTS_ID = \"%s\"" % wml_remote_training_system_one_asset_uid)

As the Admin, you have now launched a Federated Learning experiment. Copy the output from the previous cell. Open Part 2 - WML Federated Learning with MNIST for Party and paste the output into the first code cell.

# <hr>
Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.
 <br><br>
<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>